<h2>Importando bibliotecas</h2>

In [1]:
import findspark
findspark.init()
import pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
import pandas as pd
import pyspark.pandas as ps

<h2> Criando sessão do spark + configurações de conexão com bucket </h2>

In [4]:
spark = SparkSession.builder.appName('Read CSV File - Convert to Parquet').getOrCreate()

22/04/02 09:20:25 WARN Utils: Your hostname, moon resolves to a loopback address: 127.0.1.1; using 192.168.0.185 instead (on interface wlo1)
22/04/02 09:20:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 09:20:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.EnvironmentVariableCredentialsProvider')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://localhost:9000')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')
    
load_config(spark.sparkContext)


<h2> Lendo dados do bucket </h2>

In [22]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType

reviews_schema = StructType(
    [StructField("_c0", IntegerType(), False),
     StructField("app_id", IntegerType(), False),
     StructField("app_name", StringType(), False),
     StructField("review_id", IntegerType(), False),
     StructField("language", IntegerType(), False),
     StructField("timestamp_created", IntegerType(), False),
     StructField("timestamp_updated", IntegerType(), False),
     StructField("recommended", StringType(), False),
     StructField("votes_helpful", IntegerType(), False),
     StructField("votes_funny", IntegerType(), False),
     StructField("weighted_vote_score", IntegerType(), False),
     StructField("comment_count", IntegerType(), False),
     StructField("steam_purchase", StringType(), False),
     StructField("received_for_free", StringType(), False),
     StructField("written_during_early_access", StringType(), False),
     StructField("author.steamid", IntegerType(), False),
     StructField("author.num_games_owned", IntegerType(), False),
     StructField("author.num_reviews", IntegerType(), False),
     StructField("author.playtime_forever", IntegerType(), False),
     StructField("author.playtime_last_two_weeks", IntegerType(), False),
     StructField("author.playtime_at_review", IntegerType(), False),
     StructField("author.last_played", IntegerType(), False)])

In [24]:
df = spark.read.csv('s3a://bronze/dataset/*', multiLine=True, header=True, schema=reviews_schema) 

In [25]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: integer (nullable = true)
 |-- timestamp_created: integer (nullable = true)
 |-- timestamp_updated: integer (nullable = true)
 |-- recommended: string (nullable = true)
 |-- votes_helpful: integer (nullable = true)
 |-- votes_funny: integer (nullable = true)
 |-- weighted_vote_score: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- written_during_early_access: string (nullable = true)
 |-- author.steamid: integer (nullable = true)
 |-- author.num_games_owned: integer (nullable = true)
 |-- author.num_reviews: integer (nullable = true)
 |-- author.playtime_forever: integer (nullable = true)
 |-- author.playtime_last_two_weeks: integer (nullable = true)
 |-- author.playtime_at_review: integer 

In [26]:
print(f"Number of records = {df.count():,.0f}")

Number of records = 21,947,105


In [28]:
# Recomendo coluna de index

df.drop('_c0')\
    .printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: integer (nullable = true)
 |-- timestamp_created: integer (nullable = true)
 |-- timestamp_updated: integer (nullable = true)
 |-- recommended: string (nullable = true)
 |-- votes_helpful: integer (nullable = true)
 |-- votes_funny: integer (nullable = true)
 |-- weighted_vote_score: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- written_during_early_access: string (nullable = true)
 |-- author.steamid: integer (nullable = true)
 |-- author.num_games_owned: integer (nullable = true)
 |-- author.num_reviews: integer (nullable = true)
 |-- author.playtime_forever: integer (nullable = true)
 |-- author.playtime_last_two_weeks: integer (nullable = true)
 |-- author.playtime_at_review: integer (nullable = true)
 |-- author.last_p

## Salvando dados tratados no Silver

In [29]:
df.write.partitionBy('app_id').mode('overwrite').parquet('s3a://silver/steam_reviews/reviews.parquet')

22/04/02 10:02:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 23, schema size: 22
CSV file: s3a://bronze/dataset/steam_reviews.csv
